In [1]:
import pandas as pd
df0 = pd.read_csv('master.txt')

print(df0.shape)
df0.head()

(10076, 19)


,player,date,team,home,opp,game,week,day,completions,passatt,passyards,passtds,ints,sacks,sackyards,rushatt,rushyards,rushtds,fumbles
0,Geno Smith,2013-12-01,NYJ,1,MIA,12,13,Sun,4,10,29,0,1,1.0,8.0,1,2,0,0
1,Ryan Tannehill,2013-12-01,MIA,0,NYJ,12,13,Sun,28,43,331,2,1,1.0,3.0,3,22,0,0
2,Brandon Weeden,2013-12-01,CLE,1,JAX,12,13,Sun,24,40,370,3,2,3.0,28.0,2,5,0,2
3,Joe Flacco,2013-11-28,BAL,1,PIT,12,13,Thu,24,35,251,1,0,2.0,14.0,4,7,0,1
4,Matt Flynn,2013-11-28,GNB,0,DET,12,13,Thu,10,20,139,0,1,7.0,37.0,2,4,0,2


In [2]:
import numpy as np

def sanitize(df):
    df = df.copy()
    
    # exclude postseason games
    df = df[df['game'] < 17]
    
    # remove players with fewer than 100 games
    vc = (df['player'].value_counts() >= 100).to_dict()
    enough = [p for p in vc if vc[p]]
    df = df[df['player'].isin(enough)]
    
    # fill missing values
    df['sacks'] = df['sacks'].fillna(0)
    df['sackyards'] = df['sackyards'].fillna(0)
    
    # some player names have a trailing *
    df['player'] = df['player'].str.replace(r'\*$', '')
    
    # engineer a season-year feature
    df['date'] = pd.to_datetime(df['date'], infer_datetime_format=True)
    df['season'] = df['date'].apply(lambda x: x.year if x.month > 6 else x.year - 1)
    
    # drop some columns
    ignore = ['date', 'team', 'home', 'opp', 'game', 'week', 'day']
    df = df[df.columns.drop(ignore)]
    
    return df;

df1 = sanitize(df0)
print(df1.shape)
df1.head()

(3748, 13)


,player,completions,passatt,passyards,passtds,ints,sacks,sackyards,rushatt,rushyards,rushtds,fumbles,season
3,Joe Flacco,24,35,251,1,0,2.0,14.0,4,7,0,1,2013
7,Ben Roethlisberger,28,44,257,2,0,0.0,0.0,1,11,0,0,2013
8,Tony Romo,23,32,225,1,0,2.0,17.0,1,-1,0,1,2013
9,Matthew Stafford,22,35,330,3,2,1.0,10.0,1,8,0,1,2013
12,Tom Brady,34,50,344,3,0,3.0,20.0,2,0,0,2,2013


In [3]:
df1.to_csv('sanitized.txt', index=False)

In [3]:
df1['player'].value_counts(normalize=True)

Drew Brees            0.063501
Eli Manning           0.062166
Tom Brady             0.060832
Philip Rivers         0.058698
Ben Roethlisberger    0.058164
Carson Palmer         0.048559
Matt Ryan             0.048292
Aaron Rodgers         0.045891
Peyton Manning        0.045358
Joe Flacco            0.045358
Alex Smith            0.044290
Jay Cutler            0.040822
Tony Romo             0.040021
Matthew Stafford      0.039221
Ryan Fitzpatrick      0.038687
Matt Schaub           0.037353
Matt Hasselbeck       0.035219
Andy Dalton           0.033885
Cam Newton            0.033351
Russell Wilson        0.031750
Michael Vick          0.030416
Matt Cassel           0.029082
Brett Favre           0.029082
Name: player, dtype: float64